In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
import os 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
os.chdir("D:/Training/Academy/ML(Python)/Cases/Glass_Identification")

In [2]:
glass = pd.read_csv("Glass.csv")
dum_gls = pd.get_dummies(glass)
X = dum_gls.iloc[:,:-6]
y = dum_gls.iloc[:,-6:]

In [3]:
type(X)

pandas.core.frame.DataFrame

In [4]:
X.shape, y.shape

((214, 9), (214, 6))

In [5]:
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size = 0.3, 
                                                    random_state=24,stratify=glass['Type'])
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [6]:
type(y_train)

numpy.ndarray

In [7]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([149, 9])
torch.Size([149, 6])


In [8]:
joint_dataset = TensorDataset(X_torch.float(), y_torch.float())
torch.manual_seed(25)
data_loader = DataLoader(dataset=joint_dataset, batch_size=40, shuffle=True)

### Model Definition

In [9]:
torch.manual_seed(24)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1],out_features=7),
                      nn.ReLU(),
                      nn.Linear(in_features=7,out_features=6))

In [10]:
summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 7]              70
              ReLU-2                 [-1, 1, 7]               0
            Linear-3                 [-1, 1, 6]              48
Total params: 118
Trainable params: 118
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.1)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: True
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    weight_decay: 0.01
)

Generating predictions with default weights

In [12]:
y_pred = model(X_torch.float())
y_pred[:3]

tensor([[ 0.2614,  0.3164,  0.3231, -0.0748,  0.1002, -0.2294],
        [ 0.1804,  0.4066,  0.3279, -0.1816,  0.0779, -0.2740],
        [ 0.2611,  0.2778,  0.3225, -0.1486,  0.1013, -0.2364]],
       grad_fn=<SliceBackward0>)

In [13]:
print(y_torch.shape)

torch.Size([149, 6])


In [14]:
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([149, 6])
torch.Size([149, 6])


### Initial Log Loss

In [15]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(1.7288, grad_fn=<DivBackward1>)

In [16]:
for epoch in np.arange(0,1000):
    for i, batch in enumerate(data_loader, 1):
      # Forward pass: Compute predicted y by passing x to the model
      y_pred_prob = model(batch[0].float())

      # Compute and print loss
      loss = criterion(y_pred_prob, batch[1].float())

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()

      # perform a backward pass (backpropagation)
      loss.backward()

      # Update the parameters
      optimizer.step()
    
    if epoch%10 == 0:
          print('epoch: ', epoch+1,' train loss: ', loss.item())
    #if loss.item() < 0.1:
    #    break

epoch:  1  train loss:  1.453670859336853
epoch:  11  train loss:  0.955051064491272
epoch:  21  train loss:  0.6764795780181885
epoch:  31  train loss:  0.7425598502159119
epoch:  41  train loss:  0.6694876551628113
epoch:  51  train loss:  0.7358137965202332
epoch:  61  train loss:  0.6524606943130493
epoch:  71  train loss:  0.7050437331199646
epoch:  81  train loss:  0.7559751272201538
epoch:  91  train loss:  0.6681950092315674
epoch:  101  train loss:  0.6164489984512329
epoch:  111  train loss:  0.6911555528640747
epoch:  121  train loss:  0.7938599586486816
epoch:  131  train loss:  0.8226144909858704
epoch:  141  train loss:  0.7718819379806519
epoch:  151  train loss:  0.5795518755912781
epoch:  161  train loss:  0.6619747281074524
epoch:  171  train loss:  0.7639594674110413
epoch:  181  train loss:  0.6423954367637634
epoch:  191  train loss:  0.5413957238197327
epoch:  201  train loss:  0.9434641003608704
epoch:  211  train loss:  0.5631828308105469
epoch:  221  train loss

### Training Set Log Loss

In [17]:
loss

tensor(0.6026, grad_fn=<DivBackward1>)

In [18]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [19]:
lin_output = model(X_torch_test.float()) # Equivalent predict_proba / predict
type(lin_output)

torch.Tensor

In [20]:
lin_output.size()

torch.Size([65, 6])

In [21]:
y_wt_sum = model(X_torch_test.float()) 
softmax = nn.Softmax(dim=1)
pred_proba = softmax(y_wt_sum)
pred_proba

tensor([[2.8269e-01, 2.5506e-03, 6.1968e-04, 2.6609e-03, 6.6544e-01, 4.6037e-02],
        [1.8201e-23, 2.6691e-10, 4.3217e-04, 9.9954e-01, 2.9051e-05, 3.7434e-25],
        [6.7126e-01, 1.7225e-01, 4.3006e-05, 8.8332e-06, 1.4344e-07, 1.5644e-01],
        [7.1622e-01, 1.2370e-01, 6.9685e-05, 2.0506e-05, 1.0416e-06, 1.5999e-01],
        [8.1666e-01, 2.5610e-02, 4.8084e-04, 6.6616e-04, 4.6072e-03, 1.5197e-01],
        [4.8988e-01, 3.8153e-01, 9.7262e-06, 7.2895e-07, 4.8368e-10, 1.2857e-01],
        [5.0755e-08, 3.4288e-04, 2.2795e-02, 6.6117e-01, 3.1569e-01, 3.8458e-09],
        [1.4818e-01, 8.1746e-01, 5.4930e-05, 8.8275e-06, 1.7513e-09, 3.4289e-02],
        [6.5982e-01, 1.8490e-01, 3.8522e-05, 7.3043e-06, 9.2043e-08, 1.5523e-01],
        [4.6498e-01, 4.1130e-01, 8.0713e-06, 5.3888e-07, 2.4781e-10, 1.2371e-01],
        [2.3563e-20, 1.2703e-10, 8.5657e-05, 9.9806e-01, 1.8559e-03, 6.3374e-22],
        [2.3349e-01, 6.7418e-01, 1.4064e-02, 3.8321e-02, 2.4522e-03, 3.7486e-02],
        [9.6952e

`detach().numpy()` converts `torch.Tensor` into `numpy` object

In [22]:
y_pred = np.argmax(pred_proba.detach().numpy(), axis=1 )
y_pred

array([4, 3, 0, 0, 0, 0, 3, 1, 0, 0, 3, 1, 1, 0, 1, 3, 3, 0, 2, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 3, 0, 4, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       3, 3, 0, 1, 1, 2, 0, 0, 3, 0, 1, 0, 3, 0, 1, 1, 0, 0, 2, 0, 0])

In [23]:
y_test_org = np.argmax(y_test, axis=1 )

### Test Set Accuracy Score

In [24]:
print(accuracy_score(y_test_org,y_pred))

0.676923076923077


### Test Set Log Loss

In [25]:

log_loss(y_test_org, pred_proba.detach().numpy())

1.106654460493791